## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,22.89,71,100,7.83,overcast clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,41.16,73,100,2.15,overcast clouds
2,2,Cam Ranh,VN,11.9214,109.1591,76.96,94,40,3.44,light rain
3,3,Longyearbyen,SJ,78.2186,15.6401,42.64,81,75,8.05,broken clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,44.71,49,75,21.85,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 84


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cam Ranh,VN,11.9214,109.1591,76.96,94,40,3.44,light rain
7,7,Ribeira Grande,PT,38.5167,-28.7000,72.00,88,75,8.05,broken clouds
8,8,Fort Smith,US,35.3859,-94.3986,81.18,44,1,8.05,clear sky
10,10,Puerto Ayora,EC,-0.7393,-90.3518,71.91,83,94,10.11,overcast clouds
16,16,Ahmadpur,IN,18.7000,76.9333,72.95,94,100,5.70,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                288
City                   288
Country                286
Lat                    288
Lng                    288
Max Temp               288
Humidity               288
Cloudiness             288
Wind Speed             288
Current Description    288
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty row
clean_df = preferred_cities_df.copy()
clean_df.dropna(inplace = True)
clean_df.count()

City_ID                286
City                   286
Country                286
Lat                    286
Lng                    286
Max Temp               286
Humidity               286
Cloudiness             286
Wind Speed             286
Current Description    286
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cam Ranh,VN,76.96,light rain,11.9214,109.1591,
7,Ribeira Grande,PT,72.00,broken clouds,38.5167,-28.7000,
8,Fort Smith,US,81.18,clear sky,35.3859,-94.3986,
10,Puerto Ayora,EC,71.91,overcast clouds,-0.7393,-90.3518,
16,Ahmadpur,IN,72.95,heavy intensity rain,18.7000,76.9333,
17,Jakarta,ID,83.52,few clouds,-6.2146,106.8451,
25,Lagoa,PT,71.02,overcast clouds,39.0500,-27.9833,
26,Muisne,EC,77.63,overcast clouds,0.6000,-80.0333,
27,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
29,Kapaa,US,77.32,light rain,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.dropna(inplace = True)
clean_hotel_df.count()

City                   286
Country                286
Max Temp               286
Current Description    286
Lat                    286
Lng                    286
Hotel Name             286
dtype: int64

In [10]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cam Ranh,VN,76.96,light rain,11.9214,109.1591,OYO 598 Peony Hotel
7,Ribeira Grande,PT,72.00,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
8,Fort Smith,US,81.18,clear sky,35.3859,-94.3986,Hampton Inn Fort Smith
10,Puerto Ayora,EC,71.91,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Ahmadpur,IN,72.95,heavy intensity rain,18.7000,76.9333,Hengane (Pitashri Niwas)


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))